In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:24pt;}
div.text_cell_render.rendered_html{font-size:20pt;}
div.text_cell_render li, div.text_cell_render p, code{font-size:22pt; line-height:40px;}
div.output {font-size:24pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:24pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:24pt;padding:5px;}
table.dataframe{font-size:24px;}
</style>
"""))

# [RAG 절차]

1. 문서를 읽는다
    %pip install --upgrade --quiet docx2txt
2. 문서를 쪼갠다
    %pip install -qU langchain-text-splitters
3. 쪼갠 문서를 임베딩하여 vector database에 넣음(local에 저장) cf. 클라우드에 저장
    %pip install -q langchain-chroma
4. 질문을 이용해 유사도 검색
5. 유사도 검색한 문서를 LLM에 질문과 함께 전달하여 답변 얻음(렝체인 사용 가능)
    %pip install -q langchain
    (https://smith.langchain.com에서 key생성 .env에 LANGCHAIN_API_KEY로 추가)

# 0. 패키지 설치

In [ ]:
# 문서읽어오기
%pip install --upgrade --quiet docx2txt

In [2]:
# 텍스트를 chunk로 나누는 기능만 있는 경량 모듈
%pip install -qU langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.


In [3]:
# 벡터DB(로컬DB) 어제의 chromadb가 아님
%pip install -q langchain-chroma

Note: you may need to restart the kernel to use updated packages.


In [4]:
# langchain 사용
%pip install -q langchain

Note: you may need to restart the kernel to use updated packages.


# 1. 문서읽기(X)

In [5]:
%%time
from langchain_community.document_loaders import Docx2txtLoader
loader = Docx2txtLoader("./data/소득세법(법률)(제21065호)(20260102).docx")
document = loader.load()

CPU times: total: 4.86 s
Wall time: 5.1 s


In [7]:
len(document)

1

# 2. 문서를 쪼개면서 읽기(O)
- https://docs.langchain.com/oss/python/integrations/splitters